# Processing

i proccess this into 2 version:
1. Sentiment analysis ready: just cleaning-light without remove stopwords.
2. Advance cleaning -> for topic analysis, clustering, dll.

In [1]:
import pandas as pd

In [2]:
df1 = pd.read_csv('../../processed/concat_data.csv')
df2 = pd.read_csv('../../processed/concat_data.csv')

### slangword normalizing

In [3]:
import json
import pandas as pd

In [4]:
with open('../stopword/slangwords.txt', 'r', encoding='utf-8') as f:
    slang_dict = json.load(f)
    
kamusalay_df = pd.read_csv('../stopword/new_kamusalay.csv', encoding='latin1', header=None, names=['slang', 'baku'])
kamusalay_dict = dict(zip(kamusalay_df['slang'], kamusalay_df['baku']))

def normalize_slang(text):
    words = text.split()
    words = [slang_dict.get(w, w) for w in words]
    words = [kamusalay_dict.get(w, w) for w in words]
    return ' '.join(words)

## Proccess for sentiment

In [5]:
df1.head()

,text
0,Bukan pak Tom yang mencari kebenaran tapi kebe...
1,Di duga hukuman 4.5 tahun penjara Terhadap bap...
2,"ingat Saddam Hussein pak Anis, beliau difitnah..."
3,Innalilahi wa innailaihi rojiun. Turut berduka...
4,Yang menzalimi pak Tom segera dapat azab yang ...


In [6]:
import re
import emoji

In [7]:
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    # Ganti koma (dengan/tanpa spasi setelahnya) jadi satu spasi
    text = re.sub(r',\s*', ' ', text)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'@\w+|#\w+', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[\u0600-\u06FF]', '', text)
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    text = re.sub(r'[^\x00-\xFF]+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = emoji.replace_emoji(text, replace=' ')
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [8]:
# stemming using sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# cara pakai
def stem_text(text):
    return stemmer.stem(text)

In [9]:
df1.isnull().sum()

# drop rows with null values
df1 = df1.dropna(subset=['text'])

In [10]:
def half_clean(text):
    text = clean_text(text)
    text = normalize_slang(text)
    # text = stem_text(text)
    return text

from tqdm import tqdm
tqdm.pandas()
df1['cleaned1'] = df1['text'].progress_apply(half_clean)

100%|██████████| 7186/7186 [00:00<00:00, 8712.49it/s] 


In [11]:
df1.head()

,text,cleaned1
0,Bukan pak Tom yang mencari kebenaran tapi kebe...,bukan pak tom yang mencari kebenaran tapi kebe...
1,Di duga hukuman 4.5 tahun penjara Terhadap bap...,di duga hukuman tahun penjara terhadap bapak t...
2,"ingat Saddam Hussein pak Anis, beliau difitnah...",ingat saddam hussein pak anis beliau difitnah ...
3,Innalilahi wa innailaihi rojiun. Turut berduka...,innalilahi wa innailaihi rojiun turut berdukac...
4,Yang menzalimi pak Tom segera dapat azab yang ...,yang menzalimi pak tom segera dapat azab yang ...


In [12]:
forbidden_words = ['ngewe', 'farhan', 'gajah', 'vcs', 'tengu', 'carlina', 'Carlina', 'usman', 'wasaf']

# Fungsi cek apakah ada kata terlarang
def contains_forbidden(text):
    if not isinstance(text, str):
        return False
    return any(word in text.lower().split() for word in forbidden_words)

# Hapus baris yang mengandung kata terlarang
df1 = df1[~df1['text'].apply(contains_forbidden)]

In [13]:
df1.isnull().sum()

text        0
cleaned1    0
dtype: int64

In [14]:
df1.to_csv('../../processed/sentiment-ready.csv', index=False, encoding='utf-8-sig')

## Advance proccess

In [15]:
df2.head()

,text
0,Bukan pak Tom yang mencari kebenaran tapi kebe...
1,Di duga hukuman 4.5 tahun penjara Terhadap bap...
2,"ingat Saddam Hussein pak Anis, beliau difitnah..."
3,Innalilahi wa innailaihi rojiun. Turut berduka...
4,Yang menzalimi pak Tom segera dapat azab yang ...


In [16]:
df2.isnull().sum()

text    63
dtype: int64

In [17]:
# drop rows with null values
df2 = df2.dropna(subset=['text'])

### Pakai stopword

In [18]:
with open('../stopword/stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = set([line.strip() for line in f if line.strip()])

In [19]:
def remove_stopwords(text):
    if not isinstance(text, str):
        return ""
    words = text.split()
    filtered = [w for w in words if w not in stopwords]
    return ' '.join(filtered)

In [20]:
def full_clean(text):
    text = clean_text(text)
    text = normalize_slang(text)
    text = stem_text(text)
    text = remove_stopwords(text)
    return text

from tqdm import tqdm
tqdm.pandas()
df2['cleaned'] = df2['text'].progress_apply(full_clean)

100%|██████████| 7186/7186 [09:50<00:00, 12.16it/s] 


In [21]:
df2.head()

,text,cleaned
0,Bukan pak Tom yang mencari kebenaran tapi kebe...,pak tom cari benar benar datang tiba sabar pak...
1,Di duga hukuman 4.5 tahun penjara Terhadap bap...,duga hukum tahun penjara hadap bapak tom lembo...
2,"ingat Saddam Hussein pak Anis, beliau difitnah...",ingat saddam hussein pak anis beliau fitnah ba...
3,Innalilahi wa innailaihi rojiun. Turut berduka...,innalilahi wa innailaihi rojiun turut dukacita...
4,Yang menzalimi pak Tom segera dapat azab yang ...,zalim pak tom azab berlimpahsampai turun


In [22]:
df2 = df2[~df2['text'].apply(contains_forbidden)]

In [23]:
df2.isnull().sum()

text       0
cleaned    0
dtype: int64

In [24]:
df2.to_csv('../../processed/advanced-cleaning.csv', index=False, encoding='utf-8-sig')